# Collaborative Filtering with Graph Neural Networks

<a href="https://colab.research.google.com/github/khipu-ai/practicals-2023/blob/main/notebooks/graph_neural_networks.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Copyright

* Khipu 2023. Apache License 2.0.

### Authors

Juan Elenter, Tatiana Guevara, Ignacio Hounie,
Charilaos Kanatsoulis, Ignacio Ramírez and Alejandro Ribeiro*

*Alphabetical order.

## Introduction

The objective of this lab is to design a recommendation system that predicts the ratings that customers would give to a certain product. Say, the rating that a moviegoer would give to a specific movie, or the rating that an online shopper would give to a particular offering. To make these predictions we leverage the ratings that customers have given to this or similar products in the past. This approach to ratings predictions is called collaborative filtering. 

We model collaborative filtering as a problem that involves a graph $\mathbf{S}$ and graph signals $\mathbf{x}_u$ supported on the nodes of the graph. 
Nodes $p$ of the graph $\mathbf{S}$ represent different products and the weighted edges $S_{pq}$ denote simlarities between products $p$ and $q$. The entries of the graph signal $\mathbf{x}_u$ represent the ratings that user $u$ has given to each product. 

The motivation for developing a recommendation system is that customers have rated a subset of prodcuts. Each of us has seen a small number of movies or bought a small number of offerings. Thus, the ratings vector $\mathbf{x}_u$ contains only some entries that correspond to rated products. For the remaining entries we adopt the convention that $x_{ui}=0$. Our goal is to learn a function $\Phi(\mathbf{x}_{u}; \mathcal{H})$ that takes as input the vector $\mathbf{x}_u$ of available ratings and fills in predictions for the ratings that are not available. 

We will develop and evaluate a graph neural network (GNN) for making these rating predictions.

## Prerequisites

This practical assumes that you are familiar with:
* Python, JAX and Flax
* Basic object oriented programming
* Basic optimization (gradient descent, etc.)

## Hardware requirements

* You don't need to use a GPU (training should take minutes). However, if you want to speed up training, go to the "Runtime" menu in Colab, select "Change runtime type" and then in the popup menu, choose "GPU" in the "Hardware accelerator" box.



## Environment setup (Run cell)

Before we beging we need to import (and install) the necessary Python Packages. We use Numpy to load data, Jax for training, and matplotlib to plot and visualize results.


In [ ]:
!pip install jedi==0.17.2
!pip install --upgrade -q git+https://github.com/google/flax.git

In [ ]:
import numpy as np
from flax import linen as nn
from jax import lax, random, numpy as jnp
import copy
import pickle as pkl
import math
import matplotlib.pyplot as plt
import optax
import tqdm
from tqdm import tqdm
import jax


<hr>

# PART 1 - Collaborative Filtering

<hr>



## Movie Rating Data: Graph, Inputs, and Outputs

As a specific example, we use the MovieLens-100k dataset. The MovieLens-100k dataset consists of ratings given by $U$ users to $P$ movies (products). The existing movie ratings are integer values between 1 and 5. Therefore, the data are represented by a $U\times P$ matrix $X$ where $x_{up}$ is the rating that user $u$ gives to movie $p$. If user $u$ has not rated movie $p$, then $x_{up} = 0$. We see that each row of this matrix corresponds to a vector of ratings $\mathbf{x}_u$ of a particular user.

To build the collaborative filtering system, we use the rating history of all movies to compute a graph of movie similarities. In this graph edges are crosscorrelations of movie ratings. The graph can be constructed from the raw $U\times P$ movie rating matrix, but to make matters simpler we have constructed this graph already and are making it available as part of the dataset. 


To train this collaborative filtering system we use rating histories to create a dataset with entries $(\mathbf{x}_n, y_n, p_n)$. In these entries $\mathbf{x}_n$ is a vector that contains the ratings of a particular user, $y_n$ is a scalar that contains a rating that we want to predict, and $p_n$ is the index of the movie (product) that corresponds to the rating $y_n$. To evaluate this collaborative filtering system we use rating histories to create a dataset with entries having the same format. Both of these datasets can be constructed from the raw $U\times P$ movie rating matrix, but to make matters simpler we have constructed them already and are making them available as part of the dataset. 

# Task 1 Dataset set-up

*   Step 1: Download the data from [this link](https://drive.google.com/file/d/10qoedde9D6vrnv8HO_q38khGHfcD8pk_/view?usp=share_link).
*   Step 2: Upload the file 'movie_data_numpy.p' to this Colab Session by clicking on Files>Upload in the left-side menu.

In [ ]:
# The following command loads the data. For this to work the file "movie_data.p" 
# must be uploaded to this notebook. To do that, navigate to the Jupyter Notebook 
# interface home page and use the “Upload” option. 

data = pkl.load(open("movie_data_numpy.p","rb"))

# The data file contains a graph of movie similarities. The graph has P nodes.
# This is the graph we will use to run the GNN.

S = data['S']
P = S.shape[0]

# The data file contains a training set with entries (x_n, y_n, p_n).
# These entries are stored in the arrays xTrain, yTrain, and pTrain,
# respectively. All of these arrays have nTrain columns, with each 
# column corresponding to a different entry in the dataset.

xTrain = data['train']['x'] # Available ratings for making predictions. 
yTrain = data['train']['y'] # Rating to be predicted
pTrain = data['train']['p'] # Index of the movie whose rating is being predicted
nTrain = xTrain.shape[0]


# Same structure described above with test data

xTest = data['test']['x'] 
yTest = data['test']['y'] 
pTest = data['test']['p'] 
nTest = xTest.shape[0]


## Movie Similarity Graph

To help visualize the movie recommendation problem it is instructive to plot the movie similarity network $\mathbf{S}$; this is matrix of weights where each of the entries represents a similarity score between different movies.

In the figure below each bright dot corresponds to a large entry $S(p,q)$. This denotes a pair of movies that watchers tend give similar scores to. For instance, say that when someone scores "Star Wars IV" highly, they are likely to score "Star Wars V" highly and that the converse is also true; poor scores in one correlate with poor scores in the other. The entry $S(\text{"Star Wars IV"}, \text{"Star Wars V"})$ is large.

Fainter entries $S(p,q)$ denote pairs of movies with less socre correlation. Perhaps between "Star Wars" and "Star Trek" which have overlapping but not identical fan bases. Dark entries $S(p,q)$ correspond to pairs movies with no correlation between audience scores. Say when $p$ is the index of "Star Wars" and $q$ is the index of "Little Miss Sunshine."

(Run Cell)

In [ ]:
plt.figure(1, figsize = (8,8))
plt.title('Pattern of the Movie Recommendation Adjacency Matrix')
plt.xlabel('Movie index (p)')
plt.ylabel('Movieindex (q)')
plt.imshow(S)


## Mean Squared Error for Training and Testing

If we have a function $\hat{\mathbf{y}}_n=\Phi(\mathbf{x}_{n}; \mathcal{H})$ that makes rating predictions out of availbale ratings, we can evaluate the goodness of this function with the squared loss

\begin{align}
   \ell \big(\Phi(\mathbf{x}_{n}; \mathcal{H}), y_{n} \big)
      ~=~ 
               \Big[ \big(\hat{\mathbf{y}}_n \big)_{p_n} - y_n \Big]^2
      ~=~ 
               \Big[ \big( \Phi(\mathbf{x}_{n}; \mathcal{H}) \big)_{p_n} - y_n \Big]^2.
\end{align}

Notice that in this expression the function $\hat{\mathbf{y}}_n=\Phi(\mathbf{x}_{n}; \mathcal{H})$ makes predictions for all movies. However, we isolate entry $p_n$ and compare it against the rating $y_n$. We do this, because the rating $y_n$ of movie $p_n$ is the one we have available in the training or test sets.

We remark the fact that the function $\hat{\mathbf{y}}=\Phi(\mathbf{x}; \mathcal{H})$ makes predictions for all movies is important during operation. The idea of the recommendation system is to identify the subset of products that the customer would rate highly. They are the ones that we will recommend. This is why we want a system that has a graph signal as an output even though the available dataset has scalar outputs.

## Task 2: Write a function to evaluate the training loss



In [ ]:

def movieMSELoss(yHat, y, idxMovie):

    '''
    Movie MSE loss function. This function evaluates the square of comparing yHat(p) with y. The function 
    is overloaded to accept as an input tensors with multiple movie ratings. If
    given multiple movie ratings it computes the mean squared error (the error
    averaged over the dataset)
    
    Inputs:
      yHat: jnp.ndarray
        A set of rating estimates. It includes estimates of all movies.
      y: jnp.ndarray
        A specific rating of a specific movie
      idxMovie: int, jnp.ndarray
        The index of the movie whose rating is given by y

    Outputs:
      mse: jnp.ndarray
        Computed mean squared error.
    
    '''
    ## implement it
    return #mse


## Answer

In [ ]:
def movieMSELoss(yHat, y, idxMovie):

    '''
    Movie MSE loss function. This function evaluates the square of comparing yHat(p) with y. The function 
    is overloaded to accept as an input tensors with multiple movie ratings. If
    given multiple movie ratings it computes the mean squared error (the error
    averaged over the dataset)
    
    Inputs:
      yHat: jnp.ndarray
        A set of rating estimates. It includes estimates of all movies.
      y: jnp.ndarray
        A specific rating of a specific movie
      idxMovie: int, jnp.ndarray
        The index of the movie whose rating is given by y

    Outputs:
      mse: jnp.ndarray
        Computed mean squared error.
    
    '''
    # the .squeeze() method is needed for dimension match between y and yHat
    yHat = yHat.squeeze()
    # Isolate the predictions in yHat that we will use for comparison
    pred = yHat[jnp.arange(y.shape[0]), idxMovie]
    # Evaluate mean squared error
    mse = jnp.mean((pred-y)**2)
    return mse

<hr>

# PART 2 - Graph Convollutions

<hr>


## Graph Filter

One convenient model for processing graph signals is a graph _convolutional filter_. To define this operation we introduce a filter of order $K$ along with filter coefficients $\mathbf{H}_k$ that we group in the tensor $\mathcal{H}=[\mathbf{H}_0,\ldots, \mathbf{H}_{K-1}]$. A graph convolutional filter applied to the graph signal $\mathbf{X}$ is a set of polynomials in the graph shift operator (GSO) $\mathbf{S}$, i.e.,

\begin{equation}
\mathbf{\Phi}(\mathbf{X}; \mathbf{h}, \mathbf{S}) = \sum_{k=0}^{K-1} \mathbf{S}^k \mathbf{X}\, \mathbf{H}_k
\end{equation}

where the output $\mathbf{\Phi} (\mathbf{X}; \mathcal{H}, \mathbf{S})$ is also a graph signal and the learnable parameters $\mathcal{H}$ are the filter coefficients $\mathbf{H}_k$.

One advantage of graph filters is their locality. Indeed, we can define the diffusion sequence as the collection of graph signals $\mathbf{U}_k = \mathbf{S}^k \mathbf{X}$ to rewrite the filter above as $\mathbf{U} = \sum_{k=0}^{K-1} \mathbf{U}_k \mathbf{H}_k$. It is easy to see that the diffusion sequence is given by the recursion $\mathbf{U}_k = \mathbf{S}\mathbf{U}_{k-1}$ with $\mathbf{U}_0=\mathbf{X}$. Further observing that $S_{ij}\neq 0$ only when the pair $(i,j)$ is an edge of the graph, we see that the rows of the diffusion sequence satisfy

\begin{equation}
\mathbf{U}_{k}[i] \ = \ \sum_{j:(i,j)\in\mathcal{E}} S_{ij} \mathbf{U}_{k-1}[j],
\end{equation}

where $\mathcal{E}$ is the edge set of the graph. We can therefore interpret the graph filter as an operation that propagates information through adjacent nodes. This is a property that graph convolutional filters share with regular convolutional filters in time and offers motivation for their use in the processing of graph signals.

To implement a graph filter we need two components: a function that implements the shift-and-sum operation and a class that defines the graph filter as a learning architecture. Given the filter coefficients, the graph shift operator and the input graph signal, we can then implement a class for the graph filter.


# Task 3

Write a function that implements a graph filter. This function
takes as inputs the shift operator S, the filter coefficients Hk and the input
signal X. To further improve practical performance we add a bias term B
to the filter operation.

In [ ]:
def graph_filter(x, h, S, b):
    '''
    The following function defines a Graph Filter. 
    Inputs: 
      x: Input to Graph Filter (jnp.ndarray with dimensions B x G x N)
      h: Filter (jnp.ndarray with dimensions K x G x F)
      S: Graph Shift Operator (jnp.ndarray with dimensions N x N)
      b: Bias term (jnp.ndarray with dimensions F)

      Outpus:
      y: Output of Graph Filter
    '''
    # Implement convolution

    return #y
    

# Answer

In [ ]:
def graph_filter(x, h, S, b):
    '''
    The following function defines a Graph Filter. 
    Inputs: 
      x: Input to Graph Filter
      h: Filter
      S: Graph Shift Operator
      b: Bias term

      Outpus:
      y: Output of Graph Filter
    '''
    # X is  B x G x N
    # S is NxN
    B, G, N = x.shape
    K, _, F = h.shape
    y = jnp.zeros((B, N, F))
    # The following for-loop is utilized to perform the graph diffusions
    for k in range(K+1):
        # sum S^k x * h_k
        # We transpose x it to get dimensions B x N x G
        # The filter has dimensions G x F
        y += jnp.matmul(x.transpose((0,2,1)), h[k])
        # diffuse signal
        x = jnp.matmul(x, S)
    # y has dims B x N x F, 
    # add bias of shape F
    y = y + b
    # transpose it to get the desired dimensions B x F x N
    y = y.transpose(0, 2, 1)
    return  y
    

# Initialisation (Run Cell)

Before starting the training procedure, we need to initialize the parameters of the graph filter. Specifically, the filter weights and bias are sampled from a uniform distribution with support in $[ -\frac{1}{\sqrt{K F_{in}}}, \frac{1}{\sqrt{K F_{in}}} ]$ with K being the order of the filter and $F_{in}$ the input dimension.

In [ ]:

def init_graph_filter(rng_key, shape):
    """
    Initialise filters coefficients using a uniform distribution.

    Input: 
    rng_key: jnp.ndarray
      Random number generator key
    shape: Union(tuple, list)
      [k, f_in, f_out] with:
        k: int order of the filter
        f_in: int filter input dimension
        f_out:  int filter output dimension

    Output: jnp.ndarray 
      Randomly initialized filter matrix
    """
    (k, f_in, f_out) = shape
    stdv = 1. / math.sqrt(f_in * k)
   
    return random.uniform(rng_key, shape=(k, f_in, f_out),
                          minval=-stdv, maxval=stdv)
    
def init_bias(rng_key, shape):
    """
    Initialise bias coefficients using a uniform distribution

    Input:
    rng_key: jnp.ndarray 
      random number generator key
    shape: Union(tuple, list) 
      [k, f_in, f_out] with:
      k: int order of the filter
      f_in: int ilter input dimension
      f_out:  int filter output dimension
    
    Output: jnp.ndarray 
      Randomly initialized bias vector
    """
    (k, f_in, f_out) = shape
    stdv = 1. / math.sqrt(f_in * k)
   
    return random.uniform(rng_key, shape=(f_out,), minval=-stdv, maxval=stdv)        
    

### Flax nn.Module (Run cell)

We wrap the graph filter in a Flax Module

In [ ]:
# We wrap the graph filter in a Flax Module

class FilterModule(nn.Module):
    k: int #order of the filter
    f_in: int # filter input dimension
    f_out: int # filter output dimension
    
    # setup method gets called on init
    def setup(self):
        # declare parameters
        # conv filters
        self.h = self.param('h',# parameter key 
                            init_graph_filter, # Initialization function
                            (self.k, self.f_in, self.f_out))  # shape info.
        # bias added to each output filter
        self.b = self.param('b',# parameter key 
                            init_bias, # Initialization function
                            (self.k, self.f_in, self.f_out))  # shape info.
    
    # Define the forward pass
    def __call__(self, x, S):
        return graph_filter(x, self.h, S, self.b)


### Computing predictions and evaluating the loss (Run Cells)

We will also define a function that, given a model object and parameters, computes predictions and evaluates the loss.

In [ ]:
def predict_and_loss(params, model, x, s, y, p):
    """
    Function that performas a forward pass (i.e: computes predictions) of a flax modeel and 
    evaluates mean squared error.

    Input:
      params: flax.core.FrozenDict
         model parameters
      x: jnp.ndarray 
        input graph signal 
      y: jnp.ndarray
         output (label) graph signal
      p: jnp.ndarray
         target movies
    Output:
      mse jnp.ndarray:
        loss
    """
    yHat = model.apply(params, x, s)
    return movieMSELoss(yHat, y, p)


The following class wraps the Filter Module class defined above in an nn.Module and adds a readout layer. This will allow us ot use a graph filter for the movie rating prediction task.

In [ ]:
class GraphFilter(nn.Module):

    k: int  # order of filters
    f: int # number of filters 
    
    # setup method gets called on init

    def setup(self):
        self.gml = FilterModule(self.k, 1, self.f)
        # add last linear layer
        self.readout = FilterModule(1, self.f, 1)
    
    # Define the forward pass
    def __call__(self, x, S):

        x = self.gml(x, S)
        # linear readout
        x = self.readout(x, S)
        return x

## Instantiate Graph filter (Run Cell)
We now instantiate a Graph filter with the following properties:

*   Filter order: 5
*   Output Features: 64


In [ ]:
# Instantiate Graph filter of order 5 with 64 output features and a readout layer.
k = 5
f = 64

model = GraphFilter(k, f)

# Training the Graph Filter

### Training method and parameters

Below we define the training parameters and specify the optimiewr. The training procedure has the following charasteristics:

*   Number of epochs (full passes over the dataset): 8
*   Initial Learning Rate: 0.01
*   Optimizer: ADAM
*   Batch Size: 20

In [ ]:
# parameters
nEpochs = 8
learningRate = 0.01
batchSize = 20

# create a random key for init
key1 = random.PRNGKey(0)

# we need to pass thw random key and
# data inputs to call to the initialisation
# bc it computes a forward pass
params =  model.init(key1, xTrain, S)

# optimizer
tx = optax.adam(learning_rate=learningRate)
opt_state = tx.init(params)

#  forward and loss
loss_grad_fn = jax.value_and_grad(predict_and_loss)

In [ ]:
# Helper variables for data loading during training.
batchIndex = np.append(np.arange(0, nTrain, batchSize), nTrain)
nBatches = int(np.ceil(nTrain/batchSize))

## Training loop

Below is the main trainig loop implementation. This is a very straightforward implementation based on a fixed number of epochs. 
For each batch, the following steps are performed:

*   Compute prediction
*   Evaluate loss
*   Compute gradient of loss wrt parameters.
*   Update parameters

## TASK 4

Train a graph filter to predict movie ratings. Plot the evolution of the training loss and
evaluate the loss in the test dataset.

In [ ]:
epoch = 0 # epoch counter
lossTrain = []
for epoch in range(nEpochs):
    # permute samples
    randomPermutation = np.random.permutation(nTrain)
    idxEpoch = [int(i) for i in randomPermutation]
    epoch_loss = 0.
    for batch in tqdm(range(nBatches)):
        # Determine batch indices
        thisBatchIndices = idxEpoch[batchIndex[batch]: batchIndex[batch+1]]
        # Get the samples in this batch
        xTrainBatch = xTrain[thisBatchIndices,:,:]
        yTrainBatch = yTrain[thisBatchIndices]
        pTrainBatch = pTrain[thisBatchIndices]
        ## compute loss and gradients using loss_grad_fn
        # loss, grads = ## IMPLEMENT
        ## optimizer step using tx.update
        # updates, opt_state = ## IMPLEMENT
        ## update parameters using optax.apply_updates
        # params = ## IMPLEMENT
        ## append loss for plotting
        #lossTrain.append(loss)
        #epoch_loss += loss
    #print(f"Epoch: {epoch+1} \t Loss {epoch_loss/nBatches:.2f} \n")


### Answer

In [ ]:

epoch = 0 # epoch counter
lossTrain = []
for epoch in range(nEpochs):
    # permute samples
    randomPermutation = np.random.permutation(nTrain)
    idxEpoch = [int(i) for i in randomPermutation]
    epoch_loss = 0.
    for batch in tqdm(range(nBatches)):
        # Determine batch indices
        thisBatchIndices = idxEpoch[batchIndex[batch]: batchIndex[batch+1]]
        # Get the samples in this batch
        xTrainBatch = xTrain[thisBatchIndices,:,:]
        yTrainBatch = yTrain[thisBatchIndices]
        pTrainBatch = pTrain[thisBatchIndices]
        # compute loss and gradients
        loss, grads = loss_grad_fn(params, model, xTrainBatch, S, yTrainBatch, pTrainBatch)
        # optimizer step
        updates, opt_state = tx.update(grads, opt_state)
        # update parameters
        params = optax.apply_updates(params, updates)
        # append loss for plotting
        lossTrain.append(loss)
        epoch_loss += loss
    print(f"Epoch: {epoch+1} \t Loss {epoch_loss/nBatches:.2f} \n")



## Training error evolution

We use matplotlib to visualize the evolution of the mean squared error during training.

In [ ]:
plt.figure(figsize = (8,8))
plt.plot(lossTrain, label = 'Train MSE') 
plt.ylabel('Mean Squared Error')
plt.xlabel('Batches')
plt.title('Graph Filter Training Loss Evolution')

## Evaluate test performance

In [ ]:
yHatTest = model.apply(params, xTest, S)
testMSE = movieMSELoss(yHatTest, yTest, pTest)
print(f"Graph Filter Test Mean Squared Error: {testMSE:.5f}")

<hr>

# PART 3 - Graph Neural Networks

<hr>

Graph neural networks (GNNs) extend graph filters by using pointwise nonlinearities which are nonlinear functions that are applied independently to each component of a vector. For a formal definition, begin by introducing a single variable function $\sigma:\mathbb{R}\to\mathbb{R}$ which we extend to the matrix function $\sigma:\mathbb{R}^{n\times q}\to\mathbb{R}^{n\times q}$ by independent application to each component. Thus, if we have $\mathbf{U} \in \mathbb{R}^{n\times q}$ the output matrix $\sigma(\mathbf{U})$ is such that

\begin{equation}
\sigma\big(\,\mathbf{U}\,\big) \ : \ \big[\,\sigma\big(\,\mathbf{U}\,\big)\,\big]_{i,j} = \sigma\big(\,\mathbf{U}_{i,j}\,\big).
\end{equation}

In a single layer GNN, the graph signal $\mathbf{U} = \sum_{k=0}^{K-1} \mathbf{U}_k \mathbf{H}_k$ is passed through a pointwise nonlinear function to produce the output

\begin{equation}
\mathbf{\Phi}(\mathbf{X};\mathcal{H}, \mathbf{S}) = \sigma(\mathbf{U}) = \sigma \left( \sum_{k=0}^{K-1} \mathbf{S}^k \mathbf{X} \mathbf{H}_k \right) \ .
\end{equation}

We say that the transform above is a graph perceptron. Different from the graph filter, the graph perceptron is a nonlinear function of the input. It is, however, a very simple form of nonlinear processing because the nonlinearity does not mix signal components. Signal components are mixed by the graph filter but are then processed element-wise through $\sigma$.


### Multi-layer GNN

Graph perceptrons can be stacked in layers to create multi-layer GNNs. This stacking is mathematically written as a function composition where the outputs of a layer become inputs to the next layer. For a formal definition let $l=1,\ldots,L$ be a layer index and $\mathbf{H}_l=\{H_{l k}\}_{k=0}^{K-1}$ be collections of $K$ graph filter coefficients associated with each layer. At layer $l$ we take as input the output $\mathbf{X}_{l-1}$ of layer $l-1$ which we process with the filter $\mathbf{\Phi} (\mathbf{C}; \mathcal{H}_l, \mathbf{S})$ to produce the intermediate feature

\begin{align}
\mathbf{U}_{l} \ = \ \sum_{k=0}^{K-1} \mathbf{S}^k\, \mathbf{X}_{l-1} \mathbf{H}_{l k},
\end{align}

where, by convention, we say that $\mathbf{X}_0 = \mathbf{X}$ so that the given graph signal $\mathbf{X}$ is the GNN input. As in the case of the graph perceptron, this feature is passed through a pointwise nonlinear function to produce the $l$th layer output

\begin{align}
\mathbf{X}_{l} \ = \ \sigma(\mathbf{U}_l )
\ = \ \sigma\Bigg(\sum_{k=0}^{K-1} \mathbf{S}^k\, \mathbf{X}_{l-1} \mathbf{H}_{l k}\Bigg) .
\end{align}

After recursive repetition of the above formula for $l=1,\ldots,L$ we reach the $L$th layer whose output $\mathbf{x}_L$ is not further processed and is declared the GNN output $\mathbf{y}=\mathbf{x}_L$. To represent the output of the GNN we define the filter tensor $\mathcal{H}:=\{\mathcal{H}_l\}_{l=1}^L$ grouping the $L$ tensors of filter coefficients at each layer, and define the operator $\mathbf{\Phi}(\,\cdot\,;\mathcal{H},\mathbf{S})$ as the map

\begin{align}
\mathbf{\Phi} (\mathbf{X}; \mathcal{H}, \mathbf{S}) = \mathbf{X}_L.
\end{align}





# Task 5
Implement a multi-layer GNN can be built using the nn.Module that takes in the number of layers, the number of filter taps in each layer and the non-linearity and instantiates as many graph filters as there are layers. The ``__call__`` recieves the input signal and adjacency matrix, and computes the output of the GNN.


In [ ]:
#@title
class GNNModule(nn.Module):
    l: int # number of layers
    k: list #(list of length n) with order of filter for each layer
    f: list #(vector of length n+1) input dimension to each layer
    sigma: nn.Module # nonlinear activation function (same for all layers)
    # setup method gets called on init
    def setup(self):
        gml = []
        ## append layers
        #for layer in range(self.l-1):
            #gml.append() ## IMPLEMENT ###
        self.gml = gml
        # add last linear layer
        #self.readout = ## IMPLEMENT
    
    # Define the forward pass
    def __call__(self, x, S):
        #for layer in self.gml:
            ## compute graph filter
            #x = ## IMPLEMENT
            ## apply non-linearity
            #x = ## IMPLEMENT
        # linear readout
        #x = ## IMPLEMENT
        return #x

## Answer

In [ ]:
class GNNModule(nn.Module):
    l: int # number of layers
    k: list #(list of length n) with order of filter for each layer
    f: list #(vector of length n+1) input dimension to each layer
    sigma: nn.Module # nonlinear activation function (same for all layers)
    # setup method gets called on init
    def setup(self):
        gml = []
        # append layers
        for layer in range(self.l-1):
            gml.append(FilterModule(self.k[layer], self.f[layer],self.f[layer+1]))
        self.gml = gml
        # add last linear layer
        self.readout = FilterModule(1,self.f[layer+1], self.f[layer+2])
    
    # Define the forward pass
    def __call__(self, x, S):
        for layer in self.gml:
            # graph filter
            x = layer(x, S)
            # non-linearity
            x = self.sigma(x)
        # linear readout
        x = self.readout(x, S)
        return x

# Training a GNN




## Task 6
Train a GNN to predict movie ratings. Plot the evolution of the
training loss and evaluate the loss in the test dataset. To obtain a good
loss we need to experiment with the number of layers and the number of
filter taps per layer.

We will use the following parameters (also try your own!)

*   Number of layers: 3
*   Filter order at each layer: 5, 5, 1
*   Input dimension at each layer: 1, 64, 32, 1
*   Non-linearity: ReLU(x) = max(0, x)

In [ ]:
### Define Parameters
#l = #number of layers
#k = # taps
#f = # filter outputs
#sigma = # non-linearity
### Instantiate GNN model
#model = GNNModule(l, k, f, sigma)

### Answer

In [ ]:
# Define Parameters
l = 3
k = [5,5,1]
f = [1, 64, 32, 1]
sigma = nn.relu

model = GNNModule(l, k, f, sigma)

### Initialise Model and Optimizer

We use the same training parameters. The following cell initialises the model and optimizer. (Run Cell)



In [ ]:
nEpochs = 8
learningRate = 0.01
batchSize = 20

# create a random key for initialization
key1 = random.PRNGKey(0)

# we need to pass the random key and
# data inputs to call to the initialisation
# becasue it computes a forward pass
params =  model.init(key1, xTrain, S)

# optimizer
tx = optax.adam(learning_rate=learningRate)
opt_state = tx.init(params)



### Training loop (Run Cell)

Below is the main trainig loop implementation. This is a very straightforward implementation based on a fixed number of epochs. 
For each batch, the following steps are performed:

*   Compute prediction
*   Evaluate loss
*   Compute gradient of loss wrt parameters.
*   Update parameters

In [ ]:
epoch = 0 # epoch counter
lossTrain = []
for epoch in range(nEpochs):
    # permute samples
    randomPermutation = np.random.permutation(nTrain)
    idxEpoch = [int(i) for i in randomPermutation]
    epoch_loss = 0.
    for batch in tqdm(range(nBatches)):
        # Determine batch indices
        thisBatchIndices = idxEpoch[batchIndex[batch]: batchIndex[batch+1]]
        # Get the samples in this batch
        xTrainBatch = xTrain[thisBatchIndices,:,:]
        yTrainBatch = yTrain[thisBatchIndices]
        pTrainBatch = pTrain[thisBatchIndices]
        # compute loss and gradients
        loss, grads = loss_grad_fn(params, model, xTrainBatch, S, yTrainBatch, pTrainBatch)
        # optimizer step
        updates, opt_state = tx.update(grads, opt_state)
        # update parameters
        params = optax.apply_updates(params, updates)
        # append loss for plotting
        lossTrain.append(loss)
        epoch_loss += loss

    print(f"Epoch: {epoch+1} \t Loss {epoch_loss/nBatches:.2f} \n")
    


## Training error evolution (Run Cell)

We use matplotlib to visualize the evolution of the mean squared error during training.

In [ ]:
plt.figure(figsize = (8,8))
plt.plot(lossTrain, label = 'Train MSE') 
plt.ylabel('Mean Squared Error')
plt.xlabel('Batches')
plt.title('GNN Training Loss Evolution')


## Evaluation

We evaluate the testing accuracy for the GNN.



In [ ]:
yHatTest = model.apply(params, xTest, S)
testMSE = movieMSELoss(yHatTest, yTest, pTest)
print(f"GNN Test Mean Squared Error: {testMSE:.5f}")

# **Feedback**

Please provide feedback that we can use to improve our practicals in the future.

In [ ]:
#@title Please run this cell to load the feedback form:
from IPython.display import HTML

HTML(
    """
<iframe
	src="https://forms.gle/AjhUiYxfLFV5gMGa7",
  width="80%"
	height="1200px" >
	Loading...
</iframe>
"""
)

<center><img src="https://github.com/khipu-ai/global-resources/raw/main/logo/khipu.png" /></center>
